In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("Linear Regression Project1").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv("G:\Downloads Ex\Python-and-Spark-for-Big-Data-master\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Linear_Regression\Ecommerce_Customers.csv", inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [8]:
for i in data.head(1)[0]:
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


###Setting Up DataFrame for Machine Learning

In [9]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [10]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [11]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership','Yearly Amount Spent'],
                           outputCol='features')

In [12]:
output = assembler.transform(data)

In [13]:
for i in output.head(1)[0]:
    print(i)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005
[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615,587.9510539684005]


In [14]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [15]:
final_data = output.select("features",'Yearly Amount Spent')

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [17]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [18]:
train_data.describe().show()
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                348|
|   mean|  497.7603046546141|
| stddev|  83.10088999737333|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                152|
|   mean| 502.87127045716835|
| stddev|   70.0078027011333|
|    min| 298.76200786180766|
|    max|  663.0748176067602|
+-------+-------------------+



In [19]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol= "Yearly Amount Spent")

In [20]:
lr_model = lr.fit(train_data)

In [21]:
#now Lets evaluate the the model against test data
test_results = lr_model.evaluate(test_data)

In [22]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.119815351557917...|
|-5.11590769747272...|
|-5.05906427861191...|
|-2.33058017329312...|
|-1.12549969344399...|
|8.753886504564434...|
|1.176658770418725...|
|-7.04858393874019...|
|-3.18323145620524...|
|1.944044925039634...|
|1.932676241267472...|
|1.023181539494544...|
|-1.93267624126747...|
|-5.11590769747272...|
|-1.58024704433046...|
|1.443822839064523...|
|-8.41282599139958...|
|1.421085471520200...|
|9.265477274311706...|
|-1.83035808731801...|
+--------------------+
only showing top 20 rows



In [24]:
test_results.rootMeanSquaredError

1.0177130952906346e-11

In [25]:
test_results.r2

1.0

In [27]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [28]:
unlabeled_data = test_data.select("features")

In [29]:
predictions = lr_model.transform(unlabeled_data)

In [30]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726163|
|[30.8364326747734...| 467.5019004269901|
|[30.8794843441274...|490.20659998485974|
|[31.0613251567161...|487.55545805790393|
|[31.1280900496166...| 557.2526867470659|
|[31.1695067987115...|427.35653080228406|
|[31.3123495994443...|463.59141802792885|
|[31.4252268808548...|  530.766718654769|
|[31.5257524169682...|443.96562680988507|
|[31.6098395733896...| 444.5455496510887|
|[31.7216523605090...| 347.7769266318707|
|[31.7366356860502...| 496.9334462555217|
|[31.8164283341993...| 501.1224915036757|
|[31.8745516945853...|  392.285244246268|
|[31.9048571310136...|473.94985742283194|
|[31.9096268275227...| 563.4460356732247|
|[31.9453957483445...| 657.0199239376603|
|[31.9480174211613...| 461.9208768928964|
|[31.9549038566348...| 439.9978799399177|
|[31.9563005605233...| 547.1259317472171|
+--------------------+------------

In [31]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))

RMSE: 1.0177130952906346e-11


In [32]:
print("MSE: {}".format(test_results.meanSquaredError))

MSE: 1.0357399443260442e-22
